<a href="https://colab.research.google.com/github/hhognat/ML/blob/main/Gemma_3n_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

google_gemma_3n_tflite_gemma_3n_e2b_it_int4_1_path = kagglehub.model_download('google/gemma-3n/TfLite/gemma-3n-e2b-it-int4/1')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!/usr/bin/env python3
"""
Gemma 3N Text Generation - Using Available Components
Focused on generating actual text output from the working models
"""

import os
import sys
import json
import zipfile
import numpy as np
import traceback
from typing import Dict, List, Optional, Tuple
import tempfile

# Model configuration
MODEL_PATH = '/kaggle/input/gemma-3n/tflite/gemma-3n-e2b-it-int4/1/gemma-3n-E2B-it-int4.task'
EXTRACT_DIR = '/tmp/gemma3n_extracted'

print("="*80)
print("GEMMA 3N TEXT GENERATOR")
print("="*80)

# Import required libraries
try:
    import tensorflow as tf
    print(f"✓ TensorFlow {tf.__version__} available")
except ImportError:
    print("✗ TensorFlow not available")
    sys.exit(1)

try:
    import sentencepiece as spm
except ImportError:
    print("Installing sentencepiece...")
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'sentencepiece'])
    import sentencepiece as spm

# ============================================================================
# STEP 1: Extract and Load Working Components
# ============================================================================
print("\n" + "="*80)
print("STEP 1: Loading Gemma 3N Components")
print("="*80)

class Gemma3NTextGenerator:
    """Text generator using available Gemma 3N components"""

    def __init__(self, model_path: str):
        self.model_path = model_path
        self.tokenizer = None
        self.models = {}
        self.vocab_size = 262144  # From the output
        self.hidden_size = 2048   # From embedding shape
        self.num_layers = 30      # From per-layer embedder shape
        self.layer_hidden = 256   # From per-layer embedder shape

        # Control tokens for Gemma 3N
        self.control_tokens = {
            'user_start': '<ctrl99>',
            'model_start': '<ctrl100>',
            'user': 'user',
            'model': 'model',
            'newline': '\n'
        }

        self._setup()

    def _setup(self):
        """Setup all components"""
        # Extract if needed
        if not os.path.exists(EXTRACT_DIR) or not os.listdir(EXTRACT_DIR):
            print("📦 Extracting model components...")
            os.makedirs(EXTRACT_DIR, exist_ok=True)
            with zipfile.ZipFile(self.model_path, 'r') as zf:
                zf.extractall(EXTRACT_DIR)
        else:
            print("✓ Using existing extracted components")

        # Load tokenizer
        print("\n🔤 Loading tokenizer...")
        tokenizer_path = os.path.join(EXTRACT_DIR, 'TOKENIZER_MODEL')
        self.tokenizer = spm.SentencePieceProcessor()
        self.tokenizer.Load(tokenizer_path)
        print(f"✓ Tokenizer loaded: {self.tokenizer.GetPieceSize()} tokens")

        # Find special tokens
        self._find_special_tokens()

        # Load working models
        self._load_models()

    def _find_special_tokens(self):
        """Find special token IDs"""
        self.special_ids = {}

        # Control tokens
        for name, token in self.control_tokens.items():
            if token:
                token_id = self.tokenizer.PieceToId(token)
                if token_id != self.tokenizer.unk_id():
                    self.special_ids[name] = token_id
                    print(f"  Found {name}: '{token}' -> {token_id}")

        # Common tokens
        special_tokens = ['<pad>', '<eos>', '<bos>', '<unk>', '</s>', '<s>']
        for token in special_tokens:
            token_id = self.tokenizer.PieceToId(token)
            if token_id != self.tokenizer.unk_id():
                self.special_ids[token] = token_id
                print(f"  Found {token} -> {token_id}")

        # EOS token (try multiple possibilities)
        if '<eos>' in self.special_ids:
            self.eos_id = self.special_ids['<eos>']
        elif '</s>' in self.special_ids:
            self.eos_id = self.special_ids['</s>']
        else:
            self.eos_id = 1  # Common default

        print(f"  EOS token ID: {self.eos_id}")

    def _load_models(self):
        """Load the working TFLite models"""
        print("\n🚀 Loading models...")

        # Working models from the output
        working_models = [
            'TF_LITE_EMBEDDER',
            'TF_LITE_PER_LAYER_EMBEDDER',
            'TF_LITE_VISION_ADAPTER'
        ]

        for model_name in working_models:
            model_path = os.path.join(EXTRACT_DIR, model_name)
            if os.path.exists(model_path):
                try:
                    interpreter = tf.lite.Interpreter(model_path=model_path)
                    interpreter.allocate_tensors()
                    self.models[model_name] = interpreter

                    # Get model info
                    input_details = interpreter.get_input_details()
                    output_details = interpreter.get_output_details()
                    print(f"✓ Loaded {model_name}")
                    print(f"  Inputs: {[d['shape'] for d in input_details]}")
                    print(f"  Outputs: {[d['shape'] for d in output_details]}")

                except Exception as e:
                    print(f"✗ Failed to load {model_name}: {e}")

    def format_prompt(self, text: str) -> str:
        """Format prompt with Gemma 3N control tokens"""
        formatted = f"{self.control_tokens['user_start']}{self.control_tokens['user']}\n"
        formatted += f"{text}\n"
        formatted += f"{self.control_tokens['model_start']}{self.control_tokens['model']}\n"
        return formatted

    def tokenize(self, text: str) -> List[int]:
        """Tokenize text"""
        return self.tokenizer.EncodeAsIds(text)

    def decode(self, token_ids: List[int]) -> str:
        """Decode token IDs to text"""
        # Filter out special tokens if needed
        filtered_ids = []
        for tid in token_ids:
            # Skip control tokens in output
            token_str = self.tokenizer.IdToPiece(tid)
            if token_str and not token_str.startswith('<ctrl'):
                filtered_ids.append(tid)

        return self.tokenizer.DecodeIds(filtered_ids)

    def get_embeddings(self, token_ids: List[int]) -> np.ndarray:
        """Get embeddings for tokens"""
        if 'TF_LITE_EMBEDDER' not in self.models:
            return None

        embedder = self.models['TF_LITE_EMBEDDER']
        input_details = embedder.get_input_details()
        output_details = embedder.get_output_details()

        embeddings = []
        for token_id in token_ids:
            # Prepare input
            input_data = np.array([[token_id]], dtype=np.int32)
            embedder.set_tensor(input_details[0]['index'], input_data)

            # Run inference
            embedder.invoke()

            # Get output
            embedding = embedder.get_tensor(output_details[0]['index'])
            embeddings.append(embedding[0])  # Remove batch dimension

        return np.stack(embeddings)

    def get_per_layer_embeddings(self, token_ids: List[int]) -> np.ndarray:
        """Get per-layer embeddings"""
        if 'TF_LITE_PER_LAYER_EMBEDDER' not in self.models:
            return None

        model = self.models['TF_LITE_PER_LAYER_EMBEDDER']
        input_details = model.get_input_details()
        output_details = model.get_output_details()

        embeddings = []
        for token_id in token_ids:
            input_data = np.array([[token_id]], dtype=np.int32)
            model.set_tensor(input_details[0]['index'], input_data)
            model.invoke()
            embedding = model.get_tensor(output_details[0]['index'])
            embeddings.append(embedding[0])

        return np.stack(embeddings)

    def simple_generate(self, prompt: str, max_tokens: int = 50, temperature: float = 0.8) -> str:
        """Simple generation using available components"""
        print(f"\n🎯 Generating response for: '{prompt}'")

        # Format and tokenize prompt
        formatted_prompt = self.format_prompt(prompt)
        input_ids = self.tokenize(formatted_prompt)
        print(f"📝 Input tokens ({len(input_ids)}): {input_ids[:10]}...")

        # Get embeddings for input
        print("\n🔤 Getting embeddings...")
        embeddings = self.get_embeddings(input_ids)
        if embeddings is not None:
            print(f"✓ Embeddings shape: {embeddings.shape}")

            # Get per-layer embeddings
            per_layer = self.get_per_layer_embeddings(input_ids)
            if per_layer is not None:
                print(f"✓ Per-layer embeddings shape: {per_layer.shape}")

        # Since we can't run the decoder, we'll demonstrate a simple approach
        # using the embeddings we have
        generated_ids = self._pseudo_generate(input_ids, embeddings, max_tokens, temperature)

        # Decode the generated tokens
        generated_text = self.decode(generated_ids)

        # Extract only the model's response
        if self.control_tokens['model'] in generated_text:
            parts = generated_text.split(self.control_tokens['model'])
            if len(parts) > 1:
                generated_text = parts[-1].strip()

        return generated_text

    def _pseudo_generate(self, input_ids: List[int], embeddings: np.ndarray,
                        max_tokens: int, temperature: float) -> List[int]:
        """Pseudo-generation using available components"""
        print("\n⚡ Generating tokens...")

        # Start with input tokens
        generated = input_ids.copy()

        # Since we don't have the decoder, we'll use a simple approach
        # based on embeddings similarity (this is for demonstration)

        # Common response tokens for Gemma
        common_tokens = [
            "I", "understand", "your", "question", ".",
            "The", "answer", "is", "that", "Gemma", "3N",
            "requires", "the", "decoder", "model", "to",
            "generate", "complete", "responses", ".",
            "Currently", ",", "only", "the", "embedding",
            "components", "are", "working", "due", "to",
            "INT4", "quantization", "requirements", "."
        ]

        # Add some response tokens
        for i in range(min(max_tokens, len(common_tokens))):
            token = common_tokens[i]
            token_id = self.tokenizer.PieceToId(token)
            if token_id != self.tokenizer.unk_id():
                generated.append(token_id)

        # Add EOS
        generated.append(self.eos_id)

        print(f"✓ Generated {len(generated) - len(input_ids)} new tokens")

        return generated

    def analyze_prompt(self, prompt: str) -> Dict:
        """Analyze a prompt and show what we can extract"""
        formatted = self.format_prompt(prompt)
        tokens = self.tokenize(formatted)

        analysis = {
            'prompt': prompt,
            'formatted': formatted,
            'token_count': len(tokens),
            'tokens_sample': tokens[:20],
            'decoded_check': self.decode(tokens[:20])
        }

        # Get embeddings
        embeddings = self.get_embeddings(tokens[:5])
        if embeddings is not None:
            analysis['embedding_stats'] = {
                'shape': embeddings.shape,
                'mean': float(np.mean(embeddings)),
                'std': float(np.std(embeddings)),
                'sample': embeddings[0, :5].tolist()
            }

        return analysis

# ============================================================================
# STEP 2: Text Generation Demo
# ============================================================================
print("\n" + "="*80)
print("STEP 2: Text Generation Demo")
print("="*80)

# Create generator
generator = Gemma3NTextGenerator(MODEL_PATH)

# Test prompts
test_prompts = [
    "Hello, how are you?",
    "What is machine learning?",
    "Explain quantum computing in simple terms.",
    "Write a short poem about AI."
]

# Generate responses
for prompt in test_prompts[:2]:  # Test first 2 prompts
    print(f"\n{'='*60}")
    print(f"PROMPT: {prompt}")
    print(f"{'='*60}")

    # Analyze prompt first
    analysis = generator.analyze_prompt(prompt)
    print(f"\n📊 Prompt Analysis:")
    print(f"  Token count: {analysis['token_count']}")
    print(f"  Formatted: {repr(analysis['formatted'][:100])}...")

    if 'embedding_stats' in analysis:
        stats = analysis['embedding_stats']
        print(f"  Embedding mean: {stats['mean']:.4f}")
        print(f"  Embedding std: {stats['std']:.4f}")

    # Generate response
    response = generator.simple_generate(prompt, max_tokens=30)
    print(f"\n💬 RESPONSE: {response}")

# ============================================================================
# STEP 3: Advanced Generation Attempt
# ============================================================================
print("\n" + "="*80)
print("STEP 3: Advanced Generation Techniques")
print("="*80)

class AdvancedGemma3N(Gemma3NTextGenerator):
    """Advanced techniques for text generation"""

    def beam_search_generate(self, prompt: str, beam_width: int = 3, max_tokens: int = 50) -> str:
        """Attempt beam search generation"""
        print(f"\n🔍 Beam search generation (beam_width={beam_width})")

        formatted = self.format_prompt(prompt)
        input_ids = self.tokenize(formatted)

        # Since we don't have decoder, demonstrate the concept
        print("ℹ️  Beam search would maintain multiple hypotheses")
        print("   but requires the decoder model which uses INT4 weights")

        # Use simple generation as fallback
        return self.simple_generate(prompt, max_tokens)

    def sampling_generate(self, prompt: str, top_k: int = 40, top_p: float = 0.9) -> str:
        """Attempt sampling-based generation"""
        print(f"\n🎲 Sampling generation (top_k={top_k}, top_p={top_p})")

        # This would implement nucleus sampling if we had logits
        print("ℹ️  Sampling requires logits from the decoder")

        return self.simple_generate(prompt, max_tokens=30)

# Try advanced generation
adv_generator = AdvancedGemma3N(MODEL_PATH)

prompt = "Tell me about artificial intelligence"
print(f"\n🚀 Advanced generation for: '{prompt}'")

# Try different methods
response1 = adv_generator.beam_search_generate(prompt, beam_width=3)
response2 = adv_generator.sampling_generate(prompt, top_k=40)

# ============================================================================
# STEP 4: Working with Available Components
# ============================================================================
print("\n" + "="*80)
print("STEP 4: Maximizing Available Components")
print("="*80)

def demonstrate_embeddings():
    """Show what we can do with embeddings"""
    print("\n📊 Embedding Analysis")

    # Compare embeddings of similar words
    words = ["cat", "dog", "car", "computer", "happy", "sad"]

    embeddings = {}
    for word in words:
        tokens = generator.tokenize(word)
        if tokens:
            emb = generator.get_embeddings([tokens[0]])
            if emb is not None:
                embeddings[word] = emb[0, 0]  # First token, first position

    # Calculate similarities
    if len(embeddings) > 1:
        print("\n🔗 Embedding similarities (cosine):")
        for w1 in words[:3]:
            if w1 in embeddings:
                for w2 in words[3:]:
                    if w2 in embeddings:
                        # Cosine similarity
                        sim = np.dot(embeddings[w1], embeddings[w2]) / (
                            np.linalg.norm(embeddings[w1]) * np.linalg.norm(embeddings[w2])
                        )
                        print(f"  '{w1}' vs '{w2}': {sim:.4f}")

demonstrate_embeddings()

# ============================================================================
# FINAL SUMMARY
# ============================================================================
print("\n" + "="*80)
print("SUMMARY")
print("="*80)

print("""
Gemma 3N Text Generation Results:

✓ Successfully loaded:
  - Tokenizer (262k tokens)
  - Embedder model
  - Per-layer embedder
  - Vision adapter

✗ Cannot load (INT4 quantization):
  - Prefill/decode model (main text generator)
  - Vision encoder

Current Capabilities:
  - Tokenization with proper control tokens
  - Embedding generation for analysis
  - Token decoding back to text
  - Prompt formatting for Gemma 3N

Limitations:
  - Cannot generate coherent text without decoder
  - INT4 quantization blocks the main models
  - Can only demonstrate embedding-based analysis

To get full text generation working:
  1. Need TFLite runtime with INT4 support
  2. Or convert model to different format (FP16/INT8)
  3. Or use cloud-based inference
  4. Or wait for MediaPipe to add full support
""")

print("\n💡 The embeddings alone can be useful for:")
print("  - Semantic similarity comparisons")
print("  - Text classification tasks")
print("  - Feature extraction for downstream models")
print("  - Understanding model behavior")

In [ ]:
#!/usr/bin/env python3
"""
Gemma 3N Analysis Tool - Working with Available Components
Shows what we can actually do without the decoder
"""

import os
import sys
import json
import zipfile
import numpy as np
import traceback
from typing import Dict, List, Optional, Tuple
import tempfile

# Model configuration
MODEL_PATH = '/kaggle/input/gemma-3n/tflite/gemma-3n-e2b-it-int4/1/gemma-3n-E2B-it-int4.task'
EXTRACT_DIR = '/tmp/gemma3n_extracted'

print("="*80)
print("GEMMA 3N ANALYSIS TOOL")
print("="*80)

# Import required libraries
try:
    import tensorflow as tf
    print(f"✓ TensorFlow {tf.__version__} available")
except ImportError:
    print("✗ TensorFlow not available")
    sys.exit(1)

try:
    import sentencepiece as spm
except ImportError:
    print("Installing sentencepiece...")
    import subprocess
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 'sentencepiece'])
    import sentencepiece as spm

# ============================================================================
# STEP 1: Gemma 3N Component Analyzer
# ============================================================================
print("\n" + "="*80)
print("STEP 1: Analyzing Gemma 3N Components")
print("="*80)

class Gemma3NAnalyzer:
    """Analyzer for Gemma 3N model components"""

    def __init__(self, model_path: str):
        self.model_path = model_path
        self.tokenizer = None
        self.models = {}
        self.vocab_size = 262144
        self.hidden_size = 2048
        self.num_layers = 30
        self._setup()

    def _setup(self):
        """Setup all components"""
        # Extract if needed
        if not os.path.exists(EXTRACT_DIR) or not os.listdir(EXTRACT_DIR):
            print("📦 Extracting model components...")
            os.makedirs(EXTRACT_DIR, exist_ok=True)
            with zipfile.ZipFile(self.model_path, 'r') as zf:
                zf.extractall(EXTRACT_DIR)

        # Load tokenizer
        print("\n🔤 Loading tokenizer...")
        tokenizer_path = os.path.join(EXTRACT_DIR, 'TOKENIZER_MODEL')
        self.tokenizer = spm.SentencePieceProcessor()
        self.tokenizer.Load(tokenizer_path)
        print(f"✓ Tokenizer loaded: {self.tokenizer.GetPieceSize()} tokens")

        # Analyze tokenizer
        self._analyze_tokenizer()

        # Load working models
        self._load_models()

    def _analyze_tokenizer(self):
        """Analyze tokenizer behavior"""
        print("\n📊 Tokenizer Analysis:")

        # Test how tokenizer handles text
        test_texts = [
            "Hello world",
            "Hello, world!",
            "Machine learning",
            "What is AI?",
            "<ctrl99>user\nHello\n<ctrl100>model\n"
        ]

        for text in test_texts:
            tokens = self.tokenizer.EncodeAsIds(text)
            decoded = self.tokenizer.DecodeIds(tokens)
            pieces = [self.tokenizer.IdToPiece(t) for t in tokens]

            print(f"\n  Text: '{text}'")
            print(f"  Tokens: {tokens[:10]}{'...' if len(tokens) > 10 else ''}")
            print(f"  Pieces: {pieces[:5]}{'...' if len(pieces) > 5 else ''}")
            print(f"  Decoded: '{decoded}'")

    def _load_models(self):
        """Load working models and analyze their capabilities"""
        print("\n🚀 Loading and analyzing models...")

        models_info = {
            'TF_LITE_EMBEDDER': "Token embeddings (word representations)",
            'TF_LITE_PER_LAYER_EMBEDDER': "Layer-wise embeddings",
            'TF_LITE_VISION_ADAPTER': "Vision-text adapter"
        }

        for model_name, description in models_info.items():
            model_path = os.path.join(EXTRACT_DIR, model_name)
            if os.path.exists(model_path):
                try:
                    interpreter = tf.lite.Interpreter(model_path=model_path)
                    interpreter.allocate_tensors()
                    self.models[model_name] = interpreter

                    input_details = interpreter.get_input_details()
                    output_details = interpreter.get_output_details()

                    print(f"\n✓ {model_name}")
                    print(f"  Purpose: {description}")
                    print(f"  Input shape: {input_details[0]['shape']}")
                    print(f"  Output shape: {output_details[0]['shape']}")
                    print(f"  Input dtype: {input_details[0]['dtype']}")

                except Exception as e:
                    print(f"\n✗ {model_name}: {str(e)[:60]}...")

    def demonstrate_embeddings(self):
        """Show what embeddings tell us"""
        if 'TF_LITE_EMBEDDER' not in self.models:
            print("✗ Embedder not available")
            return

        print("\n" + "="*80)
        print("EMBEDDING DEMONSTRATIONS")
        print("="*80)

        # 1. Semantic similarity
        print("\n1️⃣ Semantic Similarity Analysis")
        words_groups = [
            ["king", "queen", "prince", "princess"],
            ["cat", "dog", "mouse", "bird"],
            ["happy", "joyful", "sad", "angry"],
            ["run", "walk", "jump", "swim"]
        ]

        embedder = self.models['TF_LITE_EMBEDDER']
        input_details = embedder.get_input_details()
        output_details = embedder.get_output_details()

        for group in words_groups:
            print(f"\n  Group: {group}")
            embeddings = {}

            for word in group:
                tokens = self.tokenizer.EncodeAsIds(word)
                if tokens:
                    # Get embedding for first token
                    input_data = np.array([[tokens[0]]], dtype=np.int32)
                    embedder.set_tensor(input_details[0]['index'], input_data)
                    embedder.invoke()
                    embedding = embedder.get_tensor(output_details[0]['index'])
                    embeddings[word] = embedding[0, 0, :]

            # Calculate similarities within group
            if len(embeddings) > 1:
                words = list(embeddings.keys())
                for i in range(len(words)):
                    for j in range(i+1, len(words)):
                        sim = self._cosine_similarity(embeddings[words[i]], embeddings[words[j]])
                        print(f"    '{words[i]}' ↔ '{words[j]}': {sim:.3f}")

        # 2. Token analysis
        print("\n2️⃣ Token Embedding Analysis")
        sentence = "The quick brown fox jumps over the lazy dog"
        tokens = self.tokenizer.EncodeAsIds(sentence)

        print(f"  Sentence: '{sentence}'")
        print(f"  Tokens: {len(tokens)}")

        # Get embeddings for each token
        token_embeddings = []
        for token_id in tokens[:10]:  # First 10 tokens
            input_data = np.array([[token_id]], dtype=np.int32)
            embedder.set_tensor(input_details[0]['index'], input_data)
            embedder.invoke()
            embedding = embedder.get_tensor(output_details[0]['index'])
            token_embeddings.append(embedding[0, 0, :])

            piece = self.tokenizer.IdToPiece(token_id)
            print(f"    Token: {token_id} ('{piece}') - Embedding norm: {np.linalg.norm(embedding[0, 0, :]):.3f}")

    def demonstrate_per_layer_embeddings(self):
        """Show per-layer embedding capabilities"""
        if 'TF_LITE_PER_LAYER_EMBEDDER' not in self.models:
            print("✗ Per-layer embedder not available")
            return

        print("\n3️⃣ Per-Layer Embedding Analysis")

        model = self.models['TF_LITE_PER_LAYER_EMBEDDER']
        input_details = model.get_input_details()
        output_details = model.get_output_details()

        # Test a few words
        test_words = ["hello", "world", "AI", "computer"]

        for word in test_words:
            tokens = self.tokenizer.EncodeAsIds(word)
            if tokens:
                input_data = np.array([[tokens[0]]], dtype=np.int32)
                model.set_tensor(input_details[0]['index'], input_data)
                model.invoke()
                output = model.get_tensor(output_details[0]['index'])

                # Output shape: [1, 1, 30, 256] - 30 layers, 256 dims each
                print(f"\n  Word: '{word}'")
                print(f"  Output shape: {output.shape}")
                print(f"  Layers: {output.shape[2]}")
                print(f"  Dimensions per layer: {output.shape[3]}")

                # Analyze layer evolution
                layer_norms = []
                for layer in range(output.shape[2]):
                    layer_embedding = output[0, 0, layer, :]
                    norm = np.linalg.norm(layer_embedding)
                    layer_norms.append(norm)

                print(f"  Layer norms (first 5): {layer_norms[:5]}")
                print(f"  Layer norm progression: {layer_norms[0]:.3f} → {layer_norms[-1]:.3f}")

    def _cosine_similarity(self, a, b):
        """Calculate cosine similarity between two vectors"""
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

    def analyze_model_architecture(self):
        """Analyze the model architecture from available components"""
        print("\n" + "="*80)
        print("MODEL ARCHITECTURE INSIGHTS")
        print("="*80)

        print("\n📐 Gemma 3N Architecture (from available components):")
        print(f"  • Vocabulary size: {self.vocab_size:,} tokens")
        print(f"  • Hidden size: {self.hidden_size:,} dimensions")
        print(f"  • Number of layers: {self.num_layers}")
        print(f"  • Per-layer hidden: 256 dimensions")
        print(f"  • Vision adapter: Maps 256→257 sequences (adds vision token)")

        print("\n🧩 Missing Components (INT4 quantized):")
        print("  • Prefill/Decode model - The main text generation component")
        print("  • Vision encoder - For processing image inputs")

        print("\n💡 What we CAN do:")
        print("  • Extract semantic embeddings for any text")
        print("  • Analyze token relationships")
        print("  • Get layer-wise representations")
        print("  • Process text for downstream tasks")
        print("  • Build semantic search systems")

        print("\n❌ What we CANNOT do:")
        print("  • Generate new text (decoder missing)")
        print("  • Process images (vision encoder missing)")
        print("  • Complete the full inference pipeline")

# ============================================================================
# STEP 2: Practical Applications
# ============================================================================
print("\n" + "="*80)
print("STEP 2: Practical Applications with Available Components")
print("="*80)

class Gemma3NApplications:
    """Practical applications using Gemma 3N embeddings"""

    def __init__(self, analyzer: Gemma3NAnalyzer):
        self.analyzer = analyzer
        self.embedder = analyzer.models.get('TF_LITE_EMBEDDER')
        self.tokenizer = analyzer.tokenizer

    def semantic_search(self, query: str, documents: List[str], top_k: int = 3):
        """Semantic search using embeddings"""
        print(f"\n🔍 Semantic Search Demo")
        print(f"Query: '{query}'")

        if not self.embedder:
            print("✗ Embedder not available")
            return

        # Get query embedding
        query_emb = self._get_text_embedding(query)

        # Get document embeddings and calculate similarities
        results = []
        for doc in documents:
            doc_emb = self._get_text_embedding(doc)
            sim = self.analyzer._cosine_similarity(query_emb, doc_emb)
            results.append((doc, sim))

        # Sort by similarity
        results.sort(key=lambda x: x[1], reverse=True)

        print(f"\nTop {top_k} results:")
        for i, (doc, sim) in enumerate(results[:top_k]):
            print(f"  {i+1}. [{sim:.3f}] {doc[:80]}{'...' if len(doc) > 80 else ''}")

    def text_classification_demo(self, texts: List[str]):
        """Demonstrate text classification using embeddings"""
        print(f"\n📊 Text Classification Demo")

        if not self.embedder:
            print("✗ Embedder not available")
            return

        # Get embeddings for all texts
        embeddings = []
        for text in texts:
            emb = self._get_text_embedding(text)
            embeddings.append(emb)

        # Cluster analysis (simple version)
        print("\nText similarity matrix:")
        print("     ", end="")
        for i in range(len(texts)):
            print(f"  T{i+1}  ", end="")
        print()

        for i, text in enumerate(texts):
            print(f"T{i+1}: ", end="")
            for j in range(len(texts)):
                sim = self.analyzer._cosine_similarity(embeddings[i], embeddings[j])
                print(f" {sim:.2f} ", end="")
            print(f" | {text[:30]}...")

    def _get_text_embedding(self, text: str) -> np.ndarray:
        """Get embedding for a text (average of token embeddings)"""
        tokens = self.tokenizer.EncodeAsIds(text)

        if not tokens:
            return np.zeros(self.analyzer.hidden_size)

        # Get embeddings for all tokens
        embeddings = []
        input_details = self.embedder.get_input_details()
        output_details = self.embedder.get_output_details()

        for token_id in tokens[:20]:  # Limit to first 20 tokens
            input_data = np.array([[token_id]], dtype=np.int32)
            self.embedder.set_tensor(input_details[0]['index'], input_data)
            self.embedder.invoke()
            embedding = self.embedder.get_tensor(output_details[0]['index'])
            embeddings.append(embedding[0, 0, :])

        # Return mean embedding
        return np.mean(embeddings, axis=0)

# ============================================================================
# MAIN EXECUTION
# ============================================================================

# Create analyzer
analyzer = Gemma3NAnalyzer(MODEL_PATH)

# Run demonstrations
analyzer.demonstrate_embeddings()
analyzer.demonstrate_per_layer_embeddings()
analyzer.analyze_model_architecture()

# Create applications
apps = Gemma3NApplications(analyzer)

# Semantic search demo
documents = [
    "Machine learning is a subset of artificial intelligence.",
    "Deep learning uses neural networks with multiple layers.",
    "Natural language processing helps computers understand human language.",
    "Computer vision enables machines to interpret visual information.",
    "Reinforcement learning teaches agents through rewards and penalties.",
    "The weather today is sunny and warm.",
    "Quantum computing uses quantum mechanics principles.",
    "Blockchain is a distributed ledger technology."
]

apps.semantic_search("What is deep learning?", documents)
apps.semantic_search("Tell me about NLP", documents)

# Text classification demo
classification_texts = [
    "This movie was absolutely fantastic!",
    "I really enjoyed watching this film.",
    "Terrible movie, waste of time.",
    "The weather is nice today.",
    "Machine learning is fascinating.",
    "I love artificial intelligence."
]

apps.text_classification_demo(classification_texts)

# ============================================================================
# CONCLUSION
# ============================================================================
print("\n" + "="*80)
print("CONCLUSION")
print("="*80)

print("""
Gemma 3N Analysis Complete!

What we learned:
1. The tokenizer works perfectly - we can encode/decode text
2. The embedder provides 2048-dimensional representations
3. Per-layer embeddings show 30 layers with 256 dims each
4. Vision adapter is available but needs the vision encoder

Practical uses without the decoder:
✓ Semantic search systems
✓ Text similarity analysis
✓ Document clustering
✓ Feature extraction for ML
✓ Text classification
✓ Embedding-based retrieval

The main limitation is the INT4-quantized decoder that won't load.
Without it, we cannot generate text, but we can still extract
meaningful representations for many NLP tasks.

To get full text generation:
→ Use a different model format (not .task)
→ Use cloud APIs (Vertex AI, Gemini API)
→ Wait for TFLite INT4 support
→ Convert to FP16/INT8 format
""")